# PrimAITE Router Simulation Demo

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This demo uses a modified version of the ARCD Use Case 2 Network (seen below) to demonstrate the capabilities of the Network simulator in PrimAITE.

``` text
                                                                                +------------+
                                                                                |  domain_   |
                                                                   +------------+ controller |
                                                                   |            |            |
                                                                   |            +------------+
                                                                   |
                                                                   |
+------------+                                                     |            +------------+
|            |                                                     |            |            |
|  client_1  +---------+                                           |  +---------+ web_server |
|            |         |                                           |  |         |            |
+------------+         |                                           |  |         +------------+
                    +--+---------+      +------------+      +------+--+--+
                    |            |      |            |      |            |
                    |  switch_2  +------+  router_1  +------+  switch_1  |
                    |            |      |            |      |            |
                    +--+------+--+      +------------+      +--+---+--+--+
+------------+         |      |                                |   |  |         +------------+
|            |         |      |                                |   |  |         |  database  |
|  client_2  +---------+      |                                |   |  +---------+  _server   |
|            |                |                                |   |            |            |
+------------+                |                                |   |            +------------+
                              |         +------------+         |   |
                              |         |  security  |         |   |
                              +---------+   _suite   +---------+   |            +------------+
                                        |            |             |            |  backup_   |
                                        +------------+             +------------+  server    |
                                                                                |            |
                                                                                +------------+
```

## The Network
First let's create our network. The network comes 'pre-packaged' with PrimAITE in the `primaite.simulator.network.networks` module.

> ℹ️ You'll see a bunch of logs associated with parts of the Network that aren't an 'electronic' device on the Network and thus don't have a stream to log to. Soon these logs are going to be pushed to a Network Logger so we're not clogging up the PrimAITE application logs.

In [1]:
from primaite.simulator.network.networks import network_simulator_demo_example

In [2]:
network = network_simulator_demo_example()

Most of the Network components have a `.show()` function that prints a table of information about that object. We can view the Nodes and Links on the Network by calling `network.show()`.

In [3]:
network.show()

+------------------------------------------------+
|                     Nodes                      |
+-------------------+----------+-----------------+
| Node              | Type     | Operating State |
+-------------------+----------+-----------------+
| router_1          | router   | ON              |
| switch_1          | switch   | ON              |
| switch_2          | switch   | ON              |
| client_1          | computer | ON              |
| client_2          | computer | ON              |
| domain_controller | server   | ON              |
| database_server   | server   | ON              |
| web_server        | server   | ON              |
| backup_server     | server   | ON              |
| security_suite    | server   | ON              |
+-------------------+----------+-----------------+
+-----------------------------------------------------------------------------+
|                                 IP Addresses                                |
+-------------------+---

## Nodes

Now let's inspect some of the nodes. We can directly access a node on the Network by calling .`get_node_by_hostname`. Like Network, a Node, along with some core services like ARP, have a `.show()` method.

### Router Nodes

First we'll inspect the Router node and some of it's core services.

Calling `router.show()` displays the Ethernet interfaces on the Router. If you need a table in markdown format, pass `markdown=True`.

In [4]:
network.get_node_by_hostname("router_1").show()

+---------------------------------------------------------------+
|                  router_1 Network Interfaces                  |
+------+-------------------+-----------------+-------+----------+
| Port | MAC Address       | Address         | Speed | Status   |
+------+-------------------+-----------------+-------+----------+
| 1    | 29:65:23:1a:e7:de | 192.168.1.1/24  | 100.0 | Enabled  |
| 2    | 81:d0:be:ff:72:ea | 192.168.10.1/24 | 100.0 | Enabled  |
| 3    | 87:a7:66:a3:2a:f5 | 127.0.0.1/8     | 100.0 | Disabled |
| 4    | 46:5e:8f:17:cc:23 | 127.0.0.1/8     | 100.0 | Disabled |
| 5    | a0:ff:5d:4d:d7:f9 | 127.0.0.1/8     | 100.0 | Disabled |
+------+-------------------+-----------------+-------+----------+


Calling `router.arp.show()` displays the Router ARP Cache.

In [5]:
network.get_node_by_hostname("router_1").arp.show()

+--------------------------------------------------------------+
|                      router_1 ARP Cache                      |
+---------------+-------------------+-------------------+------+
| IP Address    | MAC Address       | Via               | Port |
+---------------+-------------------+-------------------+------+
| 192.168.10.21 | c9:80:02:9f:79:dd | 81:d0:be:ff:72:ea | 2    |
| 192.168.10.22 | a3:1e:2f:ff:f1:c2 | 81:d0:be:ff:72:ea | 2    |
| 192.168.1.10  | 91:91:d4:6e:a9:b7 | 29:65:23:1a:e7:de | 1    |
| 192.168.1.14  | 85:73:f5:59:04:97 | 29:65:23:1a:e7:de | 1    |
| 192.168.1.11  | dc:7a:05:64:03:b5 | 29:65:23:1a:e7:de | 1    |
| 192.168.1.16  | fc:a2:ad:82:e9:b2 | 29:65:23:1a:e7:de | 1    |
| 192.168.1.110 | d0:69:9e:88:12:3a | 29:65:23:1a:e7:de | 1    |
+---------------+-------------------+-------------------+------+


Calling `router.acl.show()` displays the Access Control List.

In [6]:
network.get_node_by_hostname("router_1").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                        router_1 Access Control List                                       |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 0     | PERMIT | ANY      | ANY    | ANY          | 5432     | ANY    | ANY          | 5432     | 0       |
| 1     | PERMIT | ANY      | ANY    | ANY          | 53       | ANY    | ANY          | 53       | 0       |
| 2     | PERMIT | ANY      | ANY    | ANY          | 21       | ANY    | ANY          | 21       | 0       |
| 3     | PERMIT | ANY      | ANY    | ANY          | 80       | ANY    | ANY          | 80       | 0       |
| 22    | 

Calling `router.router_table.show()` displays the static routes the Router provides. 

In [7]:
network.get_node_by_hostname("router_1").route_table.show()

+------------------------------------------------+
|              router_1 Route Table              |
+-------+-----------------+-------------+--------+
| Index | Address         | Next Hop    | Metric |
+-------+-----------------+-------------+--------+
| 0     | 192.168.10.0/24 | 192.168.1.2 | 0.0    |
+-------+-----------------+-------------+--------+


Calling `router.sys_log.show()` displays the Router system log. By default, only the last 10 log entries are displayed, this can be changed by passing `last_n=<number of log entries>`.

NB: For `sys_log.show()` to work correctly log files need to be created with a sys_log level of INFO or below.

In [8]:
network.get_node_by_hostname("router_1").sys_log.show()

+--------------------------------------------------------------------------------------------------------------------------------------------+
|                                                              router_1 Sys Log                                                              |
+-------------------------+-------+----------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                  |
+-------------------------+-------+----------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:02:25,661 | INFO  | Sending ARP reply from 29:65:23:1a:e7:de/192.168.1.1 to 192.168.1.11/dc:7a:05:64:03:b5                   |
| 2025-03-24 10:02:25,663 | INFO  | Forwarding frame to internally from port 1 to port 1                                                     |

### Switch Nodes

Next we'll inspect the Switch node and some of its core services.

Calling `switch.show()` displays the Switch ports on the Switch.

In [9]:
network.get_node_by_hostname("switch_1").show()

+---------------------------------------------+
|            switch_1 Switch Ports            |
+------+-------------------+-------+----------+
| Port | MAC Address       | Speed | Status   |
+------+-------------------+-------+----------+
| 1    | 66:aa:87:82:d5:0f | 100.0 | Enabled  |
| 2    | 3c:38:cd:9c:b8:8e | 100.0 | Enabled  |
| 3    | c2:58:a8:4f:c7:f2 | 100.0 | Enabled  |
| 4    | 7e:d6:98:1e:52:c2 | 100.0 | Enabled  |
| 5    | d3:7a:d8:9a:b5:9d | 100.0 | Disabled |
| 6    | df:23:39:db:52:c6 | 100.0 | Disabled |
| 7    | e9:05:71:ae:ee:96 | 100.0 | Enabled  |
| 8    | c4:54:95:f2:6b:4a | 100.0 | Enabled  |
+------+-------------------+-------+----------+


Calling `switch.sys_log.show()` displays the Switch system log. By default, only the last 10 log entries are displayed, this can be changed by passing `last_n=<number of log entries>`.

In [10]:
network.get_node_by_hostname("switch_1").sys_log.show()

+---------------------------------------------------------------------------------------+
|                                    switch_1 Sys Log                                   |
+-------------------------+-------+-----------------------------------------------------+
| Timestamp               | Level | Message                                             |
+-------------------------+-------+-----------------------------------------------------+
| 2025-03-24 10:02:25,648 | INFO  | Added MAC table entry: Port 1 -> 91:91:d4:6e:a9:b7  |
| 2025-03-24 10:02:25,649 | INFO  | Added MAC table entry: Port 8 -> 29:65:23:1a:e7:de  |
| 2025-03-24 10:02:25,653 | INFO  | Network Interface Port 3: c2:58:a8:4f:c7:f2 enabled |
| 2025-03-24 10:02:25,654 | INFO  | Added MAC table entry: Port 3 -> 85:73:f5:59:04:97  |
| 2025-03-24 10:02:25,660 | INFO  | Network Interface Port 2: 3c:38:cd:9c:b8:8e enabled |
| 2025-03-24 10:02:25,661 | INFO  | Added MAC table entry: Port 2 -> dc:7a:05:64:03:b5  |
| 2025-03-

### Computer/Server Nodes

Finally, we'll inspect a Computer or Server Node and some of its core services.

Calling `computer.show()` displays the NICs on the Computer/Server.

In [11]:
network.get_node_by_hostname("security_suite").show()

+----------------------------------------------------------------------------------+
|                      security_suite Network Interface Cards                      |
+------+------+-------------------+-------------------+-------+---------+----------+
| Port | Type | MAC Address       | Address           | Speed | Status  | NMNE     |
+------+------+-------------------+-------------------+-------+---------+----------+
| 1    | NIC  | d0:69:9e:88:12:3a | 192.168.1.110/24  | 100.0 | Enabled | Disabled |
| 2    | NIC  | b7:17:c2:8a:0e:e3 | 192.168.10.110/24 | 100.0 | Enabled | Disabled |
+------+------+-------------------+-------------------+-------+---------+----------+
+---------------------------+
| security_suite Open Ports |
+---------------------------+
| Port                      |
+---------------------------+
| 22                        |
| 53                        |
| 80                        |
| 123                       |
| 219                       |
+-------------------

Calling `computer.arp.show()` displays the Computer/Server ARP Cache.

In [12]:
network.get_node_by_hostname("security_suite").arp.show()

+------------------------------------------------------------+
|                  security_suite ARP Cache                  |
+-------------+-------------------+-------------------+------+
| IP Address  | MAC Address       | Via               | Port |
+-------------+-------------------+-------------------+------+
| 192.168.1.1 | 29:65:23:1a:e7:de | d0:69:9e:88:12:3a | 1    |
+-------------+-------------------+-------------------+------+


Calling `computer.sys_log.show()` displays the Computer/Server system log. By default, only the last 10 log entries are displayed; this can be changed by passing `last_n=<number of log entries>`.

In [13]:
network.get_node_by_hostname("security_suite").sys_log.show(last_n=25)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                     security_suite Sys Log                                                                     |
+-------------------------+---------+----------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level   | Message                                                                                                                    |
+-------------------------+---------+----------------------------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:02:25,675 | INFO    | Running Application web-browser                                                                                            |
| 2025-03-24 10:02:25,

## Basic Network Comms Check

We can perform a good old ping to check that Nodes are able to communicate with each other.

In [14]:
network.show(nodes=False, links=False)

+-----------------------------------------------------------------------------+
|                                 IP Addresses                                |
+-------------------+------+----------------+---------------+-----------------+
| Node              | Port | IP Address     | Subnet Mask   | Default Gateway |
+-------------------+------+----------------+---------------+-----------------+
| router_1          | 1    | 192.168.1.1    | 255.255.255.0 | None            |
| router_1          | 2    | 192.168.10.1   | 255.255.255.0 | None            |
| client_1          | 1    | 192.168.10.21  | 255.255.255.0 | 192.168.10.1    |
| client_2          | 1    | 192.168.10.22  | 255.255.255.0 | 192.168.10.1    |
| domain_controller | 1    | 192.168.1.10   | 255.255.255.0 | 192.168.1.1     |
| database_server   | 1    | 192.168.1.14   | 255.255.255.0 | 192.168.1.1     |
| web_server        | 1    | 192.168.1.11   | 255.255.255.0 | 192.168.1.1     |
| backup_server     | 1    | 192.168.1.1

We'll first ping client_1's default gateway.

In [15]:
network.get_node_by_hostname("client_1").ping("192.168.10.1")

True

In [16]:
network.get_node_by_hostname("client_1").sys_log.show(last_n=15)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                            client_1 Sys Log                                                                                            |
+-------------------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                                                                              |
+-------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------

Next, we'll ping the interface of the 192.168.1.0/24 Network on the Router (port 1).

In [17]:
network.get_node_by_hostname("client_1").ping("192.168.1.1")

True

And finally, we'll ping the web server.

In [18]:
network.get_node_by_hostname("client_1").ping("192.168.1.12")

False

To confirm that the ping was received and processed by the web_server, we can view the sys log

In [19]:
network.get_node_by_hostname("web_server").sys_log.show()

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                      web_server Sys Log                                                                                                                                                                                                                               

## Advanced Network Usage

We can now use the Network to perform some more advanced things.

Let's attempt to prevent client_2 from being able to ping the web server. First, we'll confirm that it can ping the server...

In [20]:
network.get_node_by_hostname("client_2").ping("192.168.1.12")

False

If we look at the client_2 sys log we can see that the four ICMP echo requests were sent and four ICMP each replies were received.

In [21]:
network.get_node_by_hostname("client_2").sys_log.show()

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                         client_2 Sys Log                                                                        |
+-------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level   | Message                                                                                                                     |
+-------------------------+---------+-----------------------------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:02:25,640 | WARNING | Cannot send ARP request as there is no outbound Network Interface to use. Try configuring the default gateway.              |
| 2025-03-24 10:

Now we'll add an ACL to block ICMP from 192.168.10.22.

In [22]:
from primaite.simulator.network.hardware.nodes.network.router import  ACLAction
from primaite.utils.validation.ip_protocol import PROTOCOL_LOOKUP

network.get_node_by_hostname("router_1").acl.add_rule(
    action=ACLAction.DENY,
    protocol=PROTOCOL_LOOKUP["ICMP"],
    src_ip_address="192.168.10.22",
    position=1
)

True

In [23]:
network.get_node_by_hostname("router_1").acl.show()

+------------------------------------------------------------------------------------------------------------------+
|                                           router_1 Access Control List                                           |
+-------+--------+----------+---------------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP        | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+---------------+--------------+----------+--------+--------------+----------+---------+
| 0     | PERMIT | ANY      | ANY           | ANY          | 5432     | ANY    | ANY          | 5432     | 0       |
| 1     | DENY   | icmp     | 192.168.10.22 | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 2     | PERMIT | ANY      | ANY           | ANY          | 21       | ANY    | ANY          | 21       | 0       |
| 3     | PERMIT | ANY      | ANY           | ANY          | 80 

Now we attempt (and fail) to ping the web server.

In [24]:
network.get_node_by_hostname("client_2").ping("192.168.1.12")

False

We can check that the ping was actually sent by client_2 by viewing the sys log.

In [25]:
network.get_node_by_hostname("client_2").sys_log.show()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                        client_2 Sys Log                                                                       |
+-------------------------+-------+-----------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                                     |
+-------------------------+-------+-----------------------------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:02:25,641 | INFO  | database-client: Configured the database-client with server_ip_address=IPv4Address('192.168.1.14'), server_password=None.   |
| 2025-03-24 10:02:25,641 | 

We can check the router sys log to see why the traffic was blocked.

In [26]:
network.get_node_by_hostname("router_1").sys_log.show()

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                             router_1 Sys Log                                                                                                            |
+-------------------------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                                                                                                               |
+-------------------------+-------+---------------------

Now a final check to ensure that client_1 can still ping the web_server.

In [27]:
network.get_node_by_hostname("client_1").ping("192.168.1.12")

False

In [28]:
network.get_node_by_hostname("client_1").sys_log.show()

+-----------------------------------------------------------------------------------------------------------------------------+
|                                                       client_1 Sys Log                                                      |
+-------------------------+-------+-------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                   |
+-------------------------+-------+-------------------------------------------------------------------------------------------+
| 2025-03-24 10:02:25,798 | INFO  | Pinging 192.168.1.1:                                                                      |
| 2025-03-24 10:02:25,799 | INFO  | Reply from 192.168.10.1: bytes=32, time=<1ms, TTL=62                                      |
| 2025-03-24 10:02:25,800 | INFO  | Reply from 192.168.10.1: bytes=32, time=<1ms, TTL=62                